In [1]:
from dotenv import load_dotenv
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset, StdioConnectionParams, StdioServerParameters
from google.genai import types

load_dotenv()

True

In [2]:
mcp_toolset = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='python3',
            args=['02_agent_mcp_server.py'],
        )
    ),
)
tools = await mcp_toolset.get_tools()

/home/user/projects/agent_101/venv/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


In [3]:
APP_NAME = "personal_assistant"
USER_ID = "default_user"
SESSION_ID = "session1234"

session_service = InMemorySessionService()
await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

Session(id='session1234', app_name='personal_assistant', user_id='default_user', state={}, events=[], last_update_time=1761285769.8514543)

In [4]:
runner = Runner(
    agent=LlmAgent(
        model=LiteLlm(model='eu.anthropic.claude-sonnet-4-5-20250929-v1:0'),
        name="claude_assistant",
        instruction="""You are an assistant that responds to user question based on the tools used. Respond politely to user's question using the output from the tools. Use Celsius scale. If the tools output is empty, then always respond with "I don't know". Use only information from the tools output to answer the question.""",
        tools=tools
    ),
    app_name=APP_NAME,
    session_service=session_service
)

In [5]:
user_message = "What's the weather in Warsaw? Is there anything interesting about this city?"
content = types.Content(role='user', parts=[types.Part(text=user_message)])

In [6]:
events = runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
response = [event.content.parts[0].text.strip() async for event in events if event.is_final_response() and event.content][-1]

In [7]:
print(response)

Based on the weather data and information from Wikipedia, here's what I can tell you about Warsaw:

## Current Weather in Warsaw:
The current temperature is **12.1°C** with moderate winds at 19.4 km/h coming from the west-northwest. The weather code indicates overcast conditions (cloudy weather).

## Interesting Facts About Warsaw:

**Historical Significance:**
- Warsaw is the capital and largest city of Poland with a population of 1.86 million residents
- The city has a dramatic history - about 85% of it was destroyed during World War II, particularly during the Warsaw Uprising in 1944
- Warsaw's Old Town was meticulously reconstructed after the war and was designated a UNESCO World Heritage Site in 1980

**Nicknames:**
- Known as the "Phoenix City" because it rose from the ashes after WWII
- Also called "Paris of the North" due to its elegant architecture

**Modern City:**
- It's home to the Varso Tower, the tallest building in the European Union
- Warsaw is a major economic hub with